    PART 2: Identify the three providers that are least similar to other providers and the the three regions that 
    are least similar to other region

### Import the necessary libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import scipy.stats as stats
import pandas_profiling

%matplotlib inline
plt.rcParams['figure.figsize'] = 10, 7.5
plt.rcParams['axes.grid'] = True

from matplotlib.backends.backend_pdf import PdfPages
from sklearn.cluster import KMeans

# center and scale the data
from sklearn.preprocessing import StandardScaler

 ### Import the dataset

In [2]:
Medicare_Provider_Charge_In = pd.read_csv('Medicare_Provider_Charge_Inpatient_DRG100_FY2011.csv')
Medicare_Provider_Charge_Out = pd.read_csv('Medicare_Provider_Charge_Outpatient_APC30_CY2011_v2.csv')

In [3]:
Medicare_Provider_Charge_In.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163065 entries, 0 to 163064
Data columns (total 12 columns):
DRG Definition                                163065 non-null object
Provider Id                                   163065 non-null int64
Provider Name                                 163065 non-null object
Provider Street Address                       163065 non-null object
Provider City                                 163065 non-null object
Provider State                                163065 non-null object
Provider Zip Code                             163065 non-null int64
Hospital Referral Region (HRR) Description    163065 non-null object
Total Discharges                              163065 non-null int64
Average Covered Charges                       163065 non-null float64
Average Total Payments                        163065 non-null float64
Average Medicare Payments                     163065 non-null float64
dtypes: float64(3), int64(3), object(6)
memory usage: 14.9+ M

In [ ]:
Medicare_Provider_Charge_Out.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43372 entries, 0 to 43371
Data columns (total 11 columns):
APC                                           43372 non-null object
Provider Id                                   43372 non-null int64
Provider Name                                 43372 non-null object
Provider Street Address                       43372 non-null object
Provider City                                 43372 non-null object
Provider State                                43372 non-null object
Provider Zip Code                             43372 non-null int64
Hospital Referral Region (HRR) Description    43372 non-null object
Outpatient Services                           43372 non-null int64
Average  Estimated Submitted Charges          43372 non-null float64
Average Total Payments                        43372 non-null float64
dtypes: float64(2), int64(3), object(6)
memory usage: 3.6+ MB


### Checking For Missing Values

In [ ]:
Medicare_Provider_Charge_In.isnull().any()

DRG Definition                                False
Provider Id                                   False
Provider Name                                 False
Provider Street Address                       False
Provider City                                 False
Provider State                                False
Provider Zip Code                             False
Hospital Referral Region (HRR) Description    False
Total Discharges                              False
Average Covered Charges                       False
Average Total Payments                        False
Average Medicare Payments                     False
dtype: bool

In [ ]:
Medicare_Provider_Charge_Out.isnull().any()

APC                                           False
Provider Id                                   False
Provider Name                                 False
Provider Street Address                       False
Provider City                                 False
Provider State                                False
Provider Zip Code                             False
Hospital Referral Region (HRR) Description    False
Outpatient Services                           False
Average  Estimated Submitted Charges          False
Average Total Payments                        False
dtype: bool

In [ ]:
pandas_profiling.ProfileReport(Medicare_Provider_Charge_In)

In [ ]:
pandas_profiling.ProfileReport(Medicare_Provider_Charge_Out)

  Based on the data profiling; I subset the columns to be used for further processing -
    
####    Columns to be used to process inpatient data-
        'DRG Definition', 'Provider Name', 'Provider State', 'Hospital Referral Region (HRR) Description',
        'Total Discharges', 'Average Covered Charges' and 'Average Total Payments'
        
####    Columns to be used to process outpatient data -
        'APC', 'Provider Name', 'Provider State', 'Hospital Referral Region (HRR) Description',
        'Outpatient Services', 'Average  Estimated Submitted Charges' and 'Average Total Payments'       

    Subsetting and renaming columns to be used for further processing

In [ ]:
Medicare_In_data = Medicare_Provider_Charge_In[['DRG Definition', 'Provider Name', 'Provider State', 'Hospital Referral Region (HRR) Description',
                                              'Total Discharges', 'Average Covered Charges', 'Average Total Payments' ]]

In [ ]:
Medicare_In_data.rename(columns={'DRG Definition' : 'Procedures', 'Hospital Referral Region (HRR) Description' : 'Region',
                                 'Total Discharges' : 'Count Of Services', 'Average Covered Charges' : 'Charges',
                                 'Average Total Payments' : 'Payment'}, inplace = True)

In [ ]:
Medicare_Out_data = Medicare_Provider_Charge_Out[['APC', 'Provider Name', 'Provider State', 'Hospital Referral Region (HRR) Description',
                                                 'Outpatient Services', 'Average  Estimated Submitted Charges', 'Average Total Payments' ]]

In [ ]:
Medicare_Out_data.rename(columns={'APC' : 'Procedures', 'Hospital Referral Region (HRR) Description' : 'Region',
                                 'Outpatient Services' : 'Count Of Services', 'Average  Estimated Submitted Charges' : 'Charges',
                                 'Average Total Payments' : 'Payment'}, inplace = True)

    Concating inpatient and Outpatient data
                               

In [ ]:
frames = [Medicare_In_data, Medicare_Out_data]
Medicare_data = pd.concat(frames)

In [ ]:
Medicare_data.head(5)

In [ ]:
Medicare_data.info()

In [ ]:
Medicare_data.isnull().any()

In [ ]:
Medicare_data['Charges'] = Medicare_data['Charges'].round(2)

In [ ]:
Medicare_data['Payment'] = Medicare_data['Payment'].round(2)

In [ ]:
Medicare_data.info()

    Grouping data by Provider and Region

    By Provider

In [ ]:
Medicare_data_group_provider = Medicare_data.groupby(['Provider Name']).agg({'Procedures' :  'nunique',
                                                                                    'Provider State' : 'nunique', 
                                                                                    'Region': 'nunique',
                                                                                    'Count Of Services': 'sum' , 
                                                                                    'Charges': 'sum' , 
                                                                                    'Payment' : 'sum'})

In [ ]:
Medicare_data_group_provider = Medicare_data_group_provider.reset_index()

In [ ]:
Medicare_data_group_provider.head()

  By Region

In [ ]:
Medicare_data_group_region = Medicare_data.groupby(['Region']).agg({'Procedures' :  'nunique',
                                                                                    'Provider State' : 'nunique', 
                                                                                    'Provider Name': 'nunique',
                                                                                    'Count Of Services': 'sum' , 
                                                                                    'Charges': 'sum' , 
                                                                                    'Payment' : 'sum'})

In [ ]:
Medicare_data_group_region = Medicare_data_group_region.reset_index()

In [ ]:
Medicare_data_group_region.head()

K Means Clustering is exploratory data analysis technique. 
    This is non-hierarchical method of grouping objects together


    Euclidean  is one of the distance measures used on K Means algorithm.
    Euclidean distance between of a observation and initial cluster centroids  1 and 2 is calculated. 
    Based on euclidean distance each observation is assigned to one of the clusters - based on minimum distance.

###    Creating the cluster at Provider level to identify the providers with least similarity

    Creating data for fitting into cluster analysis

In [ ]:
Medicare_data_group_provider_num = Medicare_data_group_provider.select_dtypes(include =['int64', 'float64']).copy()

In [ ]:
col_list_prov = Medicare_data_group_provider_num.columns

    Standardizing the data

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
sc=StandardScaler()

In [ ]:
Medicare_provider_scaled=sc.fit_transform(Medicare_data_group_provider_num)

    Applying PCA  

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
var_ratio={}
pc = PCA(n_components=3)
Medicare_provider_pca=pc.fit(Medicare_provider_scaled)
var_ratio[5]=sum(Medicare_provider_pca.explained_variance_ratio_)

In [ ]:
reduced_cr=pc.fit_transform(Medicare_provider_scaled)

In [ ]:
var_ratio

In [ ]:
reduced_cr.shape

In [ ]:
Medicare_provider_scaled.shape

In [ ]:
pd.DataFrame(pc.components_.T, columns=['PC_' + str(i) for i in range(3)], index=col_list_prov)

In [ ]:
pd.Series(pc.explained_variance_ratio_,index=['PC_' + str(i) for i in range(3)])

Clustering

#### Silhouette Coefficient

In [ ]:
from sklearn import metrics

In [ ]:
k_range = range(3, 25)
scores = []
for k in k_range:
    km = KMeans(n_clusters=k, random_state=1)
    km.fit(Medicare_provider_scaled)
    scores.append(metrics.silhouette_score(Medicare_provider_scaled, km.labels_))

In [ ]:
scores

In [ ]:
# plot the results
plt.plot(k_range, scores)
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Coefficient')
plt.grid(True)

    We will be using Silhouette Coefficient to get the most scattered cluster i.e. least coeffient 
    to get the outliers
    
    Using 22 custers for cluster analysis since it is having least coeffcient

#### For 22 cluster Solution behavior  -

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
km_22=KMeans(n_clusters=22,random_state=123)

In [ ]:
km_22.fit(reduced_cr)
km_22.labels_

In [ ]:
pd.Series(km_22.labels_).value_counts()

    From the scatter plot we can clearly see cluster 19 with 1 dot and custer 8 with 2 dots are outliers

In [ ]:
plt.figure(figsize=(7,7))
plt.scatter(reduced_cr[:,0], reduced_cr[:,1], c=km_22.labels_,cmap='Spectral',alpha=0.5)
plt.xlabel('PC_22')

In [ ]:
cluster_df_22=pd.concat([Medicare_data_group_provider, pd.Series(km_22.labels_, name='Cluster_22')], axis=1)

In [ ]:
cluster_df_22.to_csv('PART2_PROVIDER_CLUSTER.csv')

Based on the cluster analysis follwing Providers stands out as least similar to other providers by K-MEANS Analysis
        
    GOOD SAMARITAN HOSPITAL - 
                                the charges for procuders are highest as compared to other providers; 
                                also it is available at 6 states and 8 regions
                                
    CLEVELAND CLINIC and SCOTT & WHITE MEMORIAL HOSPITAL -
                                they offer highest number of procedures (APC + DRG)

### Creating the cluster at Region level to identify the region with least similarity

In [ ]:
Medicare_data_group_region.head()

    Creating data for fitting into cluster analysis

In [ ]:
Medicare_data_group_region_num = Medicare_data_group_region.select_dtypes(include =['int64', 'float64']).copy()

In [ ]:
col_list_reg = Medicare_data_group_region_num.columns

  Standardizing the data

In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
Medicare_region_scaled=sc.fit_transform(Medicare_data_group_region_num)

    Applying PCA

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
var_ratio={}
pc = PCA(n_components=4)
Medicare_region_pca=pc.fit(Medicare_region_scaled)
var_ratio[4]=sum(Medicare_region_pca.explained_variance_ratio_)

In [ ]:
reduced_cr_1=pc.fit_transform(Medicare_region_scaled)

In [ ]:
var_ratio

In [ ]:
reduced_cr_1.shape

In [ ]:
Medicare_region_scaled.shape

In [ ]:
pd.DataFrame(pc.components_.T, columns=['PC_' + str(i) for i in range(4)], index=col_list_reg)

In [ ]:
pd.Series(pc.explained_variance_ratio_,index=['PC_' + str(i) for i in range(4)])

   Clustering

#### Silhouette Coefficient

In [ ]:
from sklearn import metrics

In [ ]:
k_range = range(3, 22)
scores = []
for k in k_range:
    km = KMeans(n_clusters=k, random_state=1)
    km.fit(Medicare_region_scaled)
    scores.append(metrics.silhouette_score(Medicare_region_scaled, km.labels_))

In [ ]:
scores

In [ ]:
# plot the results
plt.plot(k_range, scores)
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Coefficient')
plt.grid(True)

    We will be using Silhouette Coefficient to get the most scattered cluster i.e. least coeffient 
    to get the outliers.
    
    Here 20 touches the least coefficent.

In [ ]:
from sklearn.cluster import KMeans

km_20=KMeans(n_clusters=20,random_state=123)
km_20.fit(reduced_cr_1)
km_20.labels_
pd.Series(km_20.labels_).value_counts()

  We see cluster 7, 8 and 17 has only one region each making it least similar to other regions -

In [ ]:
plt.figure(figsize=(7,7))
plt.scatter(reduced_cr_1[:,0], reduced_cr_1[:,1], c=km_20.labels_,cmap='Spectral',alpha=0.5)
plt.xlabel('PC_20')

In [ ]:
cluster_df_20=pd.concat([Medicare_data_group_region, pd.Series(km_20.labels_, name='Cluster_20')], axis=1)

In [ ]:
cluster_df_20.to_csv('PART2_REGION_CLUSTER.csv')

    Based on the cluster analysis follwing Regions stands out as least similar to other regions by K-MEANS Analysis
    
    CA - Los Angeles
    MA - Boston
    MD - Baltimore